In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from lib.Exercise1_1 import LQRSolver
from torch.utils.data import TensorDataset, DataLoader
import time
device = torch.device("cuda")

class DGMNN(nn.Module):
    def __init__(self):
        super(DGMNN, self).__init__()
        self.layer1 = nn.Linear(3, 100)  
        self.layer2 = nn.Linear(100, 100)
        self.layer3 = nn.Linear(100, 100)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.output = nn.Linear(100, 1)
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.tanh(self.layer2(x))
        x = self.relu(self.layer3(x))
        return self.output(x)

model = DGMNN().double().to(device)
model_DGM = nn.DataParallel(model) # 使用DataParallel包裹模


In [2]:
def get_hessian(grad, x):

    Hessian = torch.tensor([], device=device)  # 在创建时指定设备
    
    for i in range(len(x)):
        hessian = torch.tensor([], device=device)  # 在创建时指定设备
        for j in range(len(grad[i])):
            u_xxi = torch.autograd.grad(grad[i][j], x, grad_outputs=torch.ones_like(grad[i][j]), retain_graph=True, create_graph=True, allow_unused=True)[0]
            hessian = torch.cat((hessian, u_xxi[i].unsqueeze(0)))
        Hessian = torch.cat((Hessian, hessian.unsqueeze(0)), dim=0)
    return Hessian


def pde_residual(model, t, x):
    
    input = torch.cat((t.unsqueeze(1), x),dim=1)
    
    u = model(input)

    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True)[0]

    u_xx = get_hessian(u_x,x)

    residual = u_t + 0.5 * torch.einsum('bii->b', sigma @ sigma.transpose(1,2) @ u_xx) + (u_x.unsqueeze(1) @ (H @ x.unsqueeze(1).transpose(1,2)) + u_x.unsqueeze(1) @ M @ alpha + x.unsqueeze(1) @ C @ x.unsqueeze(1).transpose(1,2) + alpha.transpose(1,2) @ D @ alpha).squeeze()
    return residual

def boundary_condition(model,t, x):
    # 定义边界条件的残差
    
    T_input = T * torch.ones_like(t)

    input = torch.cat((T_input.unsqueeze(1), x),dim=1)
    u = model(input)

    return u - (x.unsqueeze(1) @ R @ x.unsqueeze(1).transpose(1,2)).squeeze()

def total_residual(model, t, x):
    # 计算内部PDE残差
    residual_loss = pde_residual(model, t, x).pow(2).mean()
    # 计算边界条件的残差
    boundary_loss = boundary_condition(model,t,x).pow(2).mean()
    
    return residual_loss + boundary_loss


In [3]:
def new_data(num_samples):
    #num_samples = 10000
    t_samples = T * torch.rand(num_samples, dtype=torch.double, requires_grad=True, device = device)
    x_ends = torch.tensor([-3,3], dtype = torch.double, device = device)
    x_samples = x_ends[0] + (x_ends[1]- x_ends[0]) * torch.rand(num_samples , 2, dtype=torch.double, requires_grad=True, device = device)
    return t_samples,x_samples

In [4]:
# Define matrices for LQR problem
H = torch.tensor([[1.2, 0.8], [-0.6, 0.9]], dtype=torch.double).to(device)
M = torch.tensor([[0.5,0.7], [0.3,1.0]], dtype=torch.double).to(device)
sigma = torch.tensor([[[0.8],[1.1]]], dtype=torch.double).to(device)
alpha = torch.tensor([[[1],[1]]], dtype=torch.double).to(device)
C = torch.tensor([[1.6, 0.0], [0.0, 1.1]], dtype=torch.double).to(device)
D = torch.tensor([[0.5, 0.0], [0.0, 0.7]], dtype=torch.double).to(device)
R = torch.tensor([[0.9, 0.0], [0.0, 1.0]], dtype=torch.double).to(device)
T = torch.tensor(1.0, dtype=torch.double).to(device)

solver = LQRSolver(H, M, sigma, C, D, R, T=T, method="euler")


In [5]:
# 准备训练
optimizer_DGM = torch.optim.Adam(model_DGM.parameters(), lr=0.001)
epoch_losses = []

Batch_size = 5
epochs = 500

# 设置残差下降的阈值
residual_threshold = 0.005
min_residual = float('inf')
for batch in range(Batch_size):
    print(f'Batch {batch+1}/{Batch_size}\n')
    
    # 假设new_data函数返回的是适合CUDA的数据，或者需要在这里将数据转移到GPU
    t_data, x_data = new_data(3000)
    t_data, x_data = t_data.to(device), x_data.to(device)  # 数据转移到GPU
    
    dataset = TensorDataset(t_data, x_data)
    dataloader = DataLoader(dataset, batch_size=350, shuffle=True)

    for epoch in range(epochs):
        model_DGM.train()
        total_loss = 0
        
        for batch_idx, (t_data_, x_data_) in enumerate(dataloader):
            t_data_, x_data_ = t_data_.to(device), x_data_.to(device)  # 数据转移到GPU
            
            optimizer_DGM.zero_grad()
            loss = total_residual(model_DGM, t_data_, x_data_) 
            loss.backward
            optimizer_DGM.step()
            total_loss += loss.item()
        epoch_losses.append(total_loss / len(dataloader))
        if len(epoch_losses) < 2:
            average_residual = abs(epoch_losses[-1] - 0)
        else:
            average_residual = abs(epoch_losses[-1] - epoch_losses[-2]) 
        if epoch == 0:
            print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')
        
        if (epoch+1) % 5 == 0:
            print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')
    # 检查残差是否下降到阈值以下
        if average_residual < residual_threshold:
            print("Residual is below threshold, starting a new training round.")
            break  # 结束当前训练轮次
        
        # 可选：更新min_residual
        if average_residual < min_residual:
            min_residual = average_residual
    print('\n')

model_DGM.eval()


Batch 1/5

Epoch 1/500 	 Loss: 157.15895540540768
Epoch 5/500 	 Loss: 156.5744525135876
Epoch 10/500 	 Loss: 157.2343503824876
Epoch 15/500 	 Loss: 157.0352401497688
Epoch 20/500 	 Loss: 157.5177927261907
Epoch 25/500 	 Loss: 158.15772361604962
Epoch 30/500 	 Loss: 157.456010234717
Epoch 35/500 	 Loss: 158.66914723811715
Epoch 40/500 	 Loss: 158.00951783855714
Residual is below threshold, starting a new training round.


Batch 2/5

Epoch 1/500 	 Loss: 159.32740969957877
Epoch 5/500 	 Loss: 158.7518501420791
Epoch 10/500 	 Loss: 158.6615372347826
Epoch 15/500 	 Loss: 158.74475132422032
Epoch 20/500 	 Loss: 158.27082690812614
Epoch 25/500 	 Loss: 158.8103050811938
Epoch 30/500 	 Loss: 158.96456274925154
Epoch 35/500 	 Loss: 158.01106209673006
Epoch 40/500 	 Loss: 159.66086135376582
Epoch 45/500 	 Loss: 159.62037069328113
Epoch 50/500 	 Loss: 159.38795703444987
Epoch 55/500 	 Loss: 158.61744021382984
Epoch 60/500 	 Loss: 157.9369798305605
Epoch 65/500 	 Loss: 159.16168516269704
Epoch 70/5

KeyboardInterrupt: 

In [ ]:
model_DGM = DGMNN().double()

# Prepare for training
optimizer_DGM = torch.optim.Adam(model_DGM.parameters(), lr=0.001)
epoch_losses = []

batch_size = 30
epochs = 500

for batch in range(batch_size):
    print(f'Batch {batch+1}/{batch_size}'+'\n')
    
    t_data,x_data = new_data(50)
    dataset = TensorDataset(t_data,x_data)
    dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

    for epoch in range(epochs):

        model_DGM.train()
        total_loss = 0
        
        for batch_idx, (t_data_,x_data_) in enumerate(dataloader):
            optimizer_DGM.zero_grad()
            loss = total_residual(model_DGM, t_data_, x_data_) 
            loss.backward(retain_graph=True)
            optimizer_DGM.step()
            total_loss += loss.item()
        epoch_losses.append(total_loss / len(dataloader))

        if epoch == 0:
            print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')
        if (epoch+1) % 100 == 0:
            print(f'Epoch {epoch+1}/{epochs} \t Loss: {total_loss / len(dataloader)}')

print('\n')
model_DGM.eval()



Batch 1/30

Epoch 1/500 	 Loss: 179.38920943757603
Epoch 100/500 	 Loss: 55.63965676485591
Epoch 200/500 	 Loss: 21.000615344534598
Epoch 300/500 	 Loss: 16.933412291149832
Epoch 400/500 	 Loss: 15.0249363441466
Epoch 500/500 	 Loss: 13.844563649177934
Batch 2/30

Epoch 1/500 	 Loss: 19.503522725018385
Epoch 100/500 	 Loss: 15.59205568977103
Epoch 200/500 	 Loss: 14.961941164535304
Epoch 300/500 	 Loss: 14.578325733578623
Epoch 400/500 	 Loss: 14.280125460164486
Epoch 500/500 	 Loss: 14.020513356986141
Batch 3/30

Epoch 1/500 	 Loss: 25.340556877811157
Epoch 100/500 	 Loss: 19.11800524720242
Epoch 200/500 	 Loss: 18.613381802375137
Epoch 300/500 	 Loss: 18.314198112088807
Epoch 400/500 	 Loss: 18.106350705380343
Epoch 500/500 	 Loss: 17.96623103348157
Batch 4/30

Epoch 1/500 	 Loss: 18.316024904658608
Epoch 100/500 	 Loss: 14.321167523864505
Epoch 200/500 	 Loss: 14.088331209657527
Epoch 300/500 	 Loss: 13.95257659499725
Epoch 400/500 	 Loss: 13.880596541372851
Epoch 500/500 	 Loss: 13

DGMNN(
  (layer1): Linear(in_features=3, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (tanh): Tanh()
  (output): Linear(in_features=50, out_features=1, bias=True)
)

In [ ]:
device = "cpu"
# Define matrices for LQR problem
H = torch.tensor([[1.2, 0.8], [-0.6, 0.9]], dtype=torch.double).to(device)
M = torch.tensor([[0.5,0.7], [0.3,1.0]], dtype=torch.double).to(device)
sigma = torch.tensor([[[0.8],[1.1]]], dtype=torch.double).to(device)
alpha = torch.tensor([[[1],[1]]], dtype=torch.double).to(device)
C = torch.tensor([[1.6, 0.0], [0.0, 1.1]], dtype=torch.double).to(device)
D = torch.tensor([[0.5, 0.0], [0.0, 0.7]], dtype=torch.double).to(device)
R = torch.tensor([[0.9, 0.0], [0.0, 1.0]], dtype=torch.double).to(device)
T = torch.tensor(1.0, dtype=torch.double).to(device)

solver = LQRSolver(H, M, sigma, C, D, R, T=T, method="euler")

In [ ]:
# Initialization
t_data,x_data = new_data(5)

value_numerical = solver.value_function(t_data.detach(),x_data.detach().unsqueeze(1))

In [ ]:
A = model_DGM(torch.cat((t_data.unsqueeze(1), x_data.squeeze(1)),dim=1))


In [ ]:
A.squeeze()

tensor([ 6.7449,  8.7295,  8.4844, 10.8576,  6.6628], device='cuda:0',
       dtype=torch.float64, grad_fn=<SqueezeBackward0>)

In [ ]:
value_numerical

tensor([ 4.6655,  1.5675,  7.7832,  9.9292, 11.0548], dtype=torch.float64)